In [ ]:
'''
llama-index                             0.12.31
llama-index-agent-openai                0.4.6
llama-index-cli                         0.4.1
llama-index-core                        0.12.31
llama-index-embeddings-huggingface      0.5.3
llama-index-embeddings-openai           0.3.1
llama-index-indices-managed-llama-cloud 0.6.11
llama-index-llms-google-genai           0.1.7
llama-index-llms-openai                 0.3.37
llama-index-multi-modal-llms-openai     0.4.3
llama-index-program-openai              0.3.1
llama-index-question-gen-openai         0.3.0
llama-index-readers-file                0.4.7
llama-index-readers-llama-parse         0.4.0
llama-index-vector-stores-qdrant        0.4.3
'''

# pip install llama-index==0.12.31
# !pip list | findstr llama-index



In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

In [2]:
try:
    response = client.get_collections()
    print("Successfully connected to Qdrant.")
    print("Available collections:", response, sep="\n")
except Exception as e:
    print("Failed to connect to Qdrant:", e)

Successfully connected to Qdrant.
Available collections:
collections=[CollectionDescription(name='corpus_halong-trained')]


In [3]:
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

True

In [5]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.google_genai import GoogleGenAI

In [6]:
embed_model = HuggingFaceEmbedding(model_name="KhoaUIT/Halong-UIT-R2GQA", max_length=512)

*NOTE*: *[Gemini](https://docs.llamaindex.ai/en/stable/examples/llm/gemini/) has largely been replaced by Google GenAI. Visit the [Google GenAI](https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/) page for the latest examples and documentation.*


* visit [model list](https://ai.google.dev/gemini-api/docs/models) to choose your favorite model
* ensure your calling API does not excceed [rate limits](https://ai.google.dev/gemini-api/docs/rate-limits)

In [ ]:
# in this notebook, we try "gemini-2.0-flash-001" first to explore agent's behavior (recommended)
# than we will change to a better model "gemini-2.5-flash-preview-04-17" to solve special cases later (the last cells of section 'Multi-Agent')

llm = GoogleGenAI(api_key=os.getenv("GEMINI_API_KEY"), model="models/gemini-2.0-flash-001", temperature=0.0) 

In [39]:
llm.metadata.is_chat_model, llm.metadata.is_function_calling_model

(True, True)

In [8]:
# QdrantVectorStore: docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/qdrant/

qdrant_vector_store = QdrantVectorStore(client=client,
                                        collection_name="corpus_halong-trained",
                                        enable_hybrid=True)

storage_context = StorageContext.from_defaults(vector_store=qdrant_vector_store)

In [9]:
# Load the index from the existing vector store

index = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store,
    storage_context=storage_context,
    embed_model=embed_model
)

# Querying a VectorStoreIndex

`index` can be used as **retriever**, **engine** and **chat_engine**

* https://docs.llamaindex.ai/en/stable/module_guides/querying/
* https://huggingface.co/learn/agents-course/en/unit2/llama-index/components#querying-a-vectorstoreindex-with-prompts-and-llms

In [10]:
query = "Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?"

In [11]:
retriever = index.as_retriever(similarity_top_k=10, vector_store_query_mode="hybrid", alpha=0.5)
nodes = retriever.retrieve(query)
context_str = "\n".join([f'{node.metadata['article']}; {node.metadata['document']}; {node.get_content()}' 
                           for id, node in enumerate(nodes)])
print(context_str)

Điều 8. Xây dựng, thẩm định học liệu điện tử; QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN; Điều 8. Xây dựng, thẩm định học liệu điện tử 1. Học liệu điện tử sử dụng trong DHTT và DHKH phải có nội dung phù hợp với đề cương môn học được phê duyệt (bao gồm chuẩn đầu ra môn học), bảo đảm tính khoa học, sư phạm và đáp ứng được nhu cầu tự học của sinh viên. 2. Học liệu điện tử được sử dụng từ nguồn sẵn có của Trường hoặc giảng viên chủ động xây dựng. 3. Khi xây dựng và sử dụng học liệu điện tử, giảng viên có trách nhiệm đảm bảo các quy định của pháp luật về Luật Sở hữu trí tuệ hiện hành. 4. Các ĐVQLMH có trách nhiệm thẩm định, đảm bảo chất lượng học liệu điện tử phục vụ DHTT, DHKH đối với các môn học mà đơn vị phụ trách. Trường có thể tổ chức thẩm định lại trong trường hợp cần thiết.
Điều 9. Sử dụng học liệu điện tử; QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN; Đ

In [ ]:
retriever = index.as_retriever(similarity_top_k=10, vector_store_query_mode="hybrid", alpha=0.5)
nodes = retriever.retrieve('cần bao nhiêu điểm toeic thì tốt nghiệp') # this query does not exist in dataset
context_str = "\n".join([f'{node.metadata['article']}; {node.metadata['document']}; {node.get_content()}' 
                           for id, node in enumerate(nodes)])
print(context_str) # (rank 1) Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp ...

Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp, QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp Để được xét tốt nghiệp sinh viên phải hoàn tất các môn học tiếng Anh trong chương trình giảng dạy tiếng Anh tại Điều 3 và có một trong các văn bằng, chứng chỉ/chứng nhận được quy định như sau: 1. Văn bằng tốt nghiệp đại học/trên đại học trong nước, nước ngoài mà ngành học hoặc ngôn ngữ sử dụng toàn phần trong đào tạo (không thông qua phiên dịch) là một trong ba ngoại ngữ: tiếng Anh, tiếng Pháp, tiếng Nhật. 2. Chứng chỉ/chứng nhận ngoại ngữ: Sinh viên có một trong các loại chứng chỉ/chứng nhận tiếng Anh, có chứng chỉ/chứng nhận tiếng Pháp, hoặc tiếng Nhật tối thiểu như sau Đối với chương trình văn bằng 2, liên thông, chương trình chuẩn phải đạt chứng chỉ tiếng Anh TOEIC Nghe-Đọc đạt từ 450 trở lên và TOEIC Nói-Viết

In [20]:
retriever = index.as_retriever(similarity_top_k=13, vector_store_query_mode="hybrid", alpha=0.5)
nodes = retriever.retrieve('điều kiện tốt nghiệp UIT với TOEIC 450 nghe đọc') # this query does not exist in dataset
context_str = "\n".join([f'{node.metadata['article']}; {node.metadata['document']}; {node.get_content()}' 
                           for id, node in enumerate(nodes)])
print(context_str) # (rank 12) Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp ...

Điều 33. Điều kiện xét tốt nghiệp và công nhận tốt nghiệp; QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN; Điều 33. Điều kiện xét tốt nghiệp và công nhận tốt nghiệp 1. Mỗi năm Trường có 4 đợt xét tốt nghiệp và cấp bằng tốt nghiệp đại học, lịch xét tốt nghiệp được ghi trong kế hoạch năm học. Để được Trường xét cấp bằng tốt nghiệp, sinh viên nộp hồ sơ đề nghị xét tốt nghiệp khi đã đáp ứng đủ các điều kiện sau đây: - Cho đến thời điểm xét tốt nghiệp, sinh viên không đang trong thời gian bị truy cứu trách nhiệm hình sự hoặc bị kỷ luật từ mức đình chỉ học tập trở lên. - Đã hoàn thành nghĩa vụ học phí. - Đã hoàn thành các môn học, tích lũy đủ số tín chỉ theo đúng chương trình đào tạo. - Đã hoàn thành các môn học Giáo dục thể chất. - Có chứng chỉ Giáo dục quốc phòng-An ninh. - Đạt chuẩn ngoại ngữ xét tốt nghiệP. - Đạt điểm rèn luyện tích lũy tối thiểu là 50 điểm. - Đã hoàn trả sách mượn cho Thư viện. 2. Căn cứ biên bản và đề nghị của Hội đ

1. `response_mode=compact` (default): similar to refining but concatenating the chunks beforehand, resulting in fewer LLM calls. <br>
    * https://huggingface.co/learn/agents-course/en/unit2/llama-index/components#response-processing
    * https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes/
    
2. `as_query_engine`: <br>
    except `llm` and `response_mode` params, you can also pass the same params of `as_retriever` 

In [15]:
#### High-Level API ####

query_engine = index.as_query_engine(llm=llm, response_mode="compact", similarity_top_k=10, vector_store_query_mode="hybrid", alpha=0.5)
query_engine.query(query)

Response(response='Trường có thể tổ chức thẩm định lại học liệu điện tử trong trường hợp cần thiết.\n', source_nodes=[NodeWithScore(node=TextNode(id_='003d642c-e402-46c8-8037-078742e04a3a', embedding=None, metadata={'article': 'Điều 8. Xây dựng, thẩm định học liệu điện tử', 'document': 'QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Điều 8. Xây dựng, thẩm định học liệu điện tử 1. Học liệu điện tử sử dụng trong DHTT và DHKH phải có nội dung phù hợp với đề cương môn học được phê duyệt (bao gồm chuẩn đầu ra môn học), bảo đảm tính khoa học, sư phạm và đáp ứng được nhu cầu tự học của sinh viên. 2. Học liệu điện tử được sử dụng từ nguồn sẵn có của Trường hoặc giảng viên chủ động xây dựng. 3. Khi xây dựng và sử dụng học liệu điện tử, giảng viên có trách nhiệm đảm bảo các quy định c

In [16]:
### Low-Level Composition API ###

from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings

# Configure global settings: LlmaIndex use OpenAI API for default settings, to see more properties, use: dir(Settings)
Settings.llm = llm

# configure retriever
custom_retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    vector_store_query_mode="hybrid",
    alpha=0.5,
    embed_model=embed_model,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query(query)
print(response)

Trường có thể tổ chức thẩm định lại học liệu điện tử trong trường hợp cần thiết.



# Create a QueryEngineTool

Using the `QueryEngineTool` class to convert `query_engine` to a tool used by an Agent

https://huggingface.co/learn/agents-course/en/unit2/llama-index/tools#creating-a-queryenginetool

In [24]:
from llama_index.core.tools import QueryEngineTool

query_engine = index.as_query_engine(llm=llm, response_mode="compact", similarity_top_k=10, vector_store_query_mode="hybrid", alpha=0.5)
query_engine_tool = QueryEngineTool.from_defaults(query_engine, name="retrieval_tool", description="use this tool to retrieve relevant documents from the database via user's query")
query_engine_tool.call(query) # check properties, use: dir(query_engine_tool)

ToolOutput(content='Trong trường hợp cần thiết, học liệu điện tử sau khi được thông qua bởi ĐVQLMH vẫn có thể được thẩm định lại.\n', tool_name='retrieval_tool', raw_input={'input': 'Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?'}, raw_output=Response(response='Trong trường hợp cần thiết, học liệu điện tử sau khi được thông qua bởi ĐVQLMH vẫn có thể được thẩm định lại.\n', source_nodes=[NodeWithScore(node=TextNode(id_='003d642c-e402-46c8-8037-078742e04a3a', embedding=None, metadata={'article': 'Điều 8. Xây dựng, thẩm định học liệu điện tử', 'document': 'QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Điều 8. Xây dựng, thẩm định học liệu điện tử 1. Học liệu điện tử sử dụng trong DHTT và DHKH phải có nội dung phù hợp với đề cương 

In [25]:
# query engine uses LLM with default prompt to generate response based on the retrieved nodes
# for more detail about the prompt, check below cell: "Accessing/Customizing Prompts within Higher-Level Modules"

from IPython.display import Markdown, display

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))


prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [28]:
'''
	Ex: create custom commonly used prompts: 
 	https://docs.llamaindex.ai/en/stable/module_guides/models/prompts/usage_pattern/#commonly-used-prompts
'''

from llama_index.core.prompts import PromptTemplate

custom_template = PromptTemplate(
    "Bạn là một trợ lý chuyên giải đáp thắc mắc của người dùng dựa vào thông tin được cung cấp dưới đây.\n\n"
    "**Tài liệu tham khảo**:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    
    "**QUY TẮC PHẢN HỒI**:\n"
    "- KHÔNG sử dụng kiến thức bên ngoài.\n"
    "- Phải **trích dẫn trực tiếp** từ tài liệu khi trả lời.\n"
    "- Sau phần trích dẫn, hãy **tóm tắt và giải thích ngắn gọn** dựa trên nội dung đó.\n\n"
    "**Câu hỏi**: {query_str}\n\n"
    
    "**Ví dụ minh hoạ**:\n"
    "- Tài liệu liên quan: điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức: ...\n"
    "- Câu trả lời mẫu: Theo điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức... Như vậy, bạn có thể chọn phương thức phù hợp với năng lực của mình. Chúc bạn thi tốt!\n\n"
)

query_engine.update_prompts({"response_synthesizer:text_qa_template": custom_template})

In [167]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Bạn là một trợ lý chuyên giải đáp thắc mắc của người dùng dựa vào thông tin được cung cấp dưới đây.

**Tài liệu tham khảo**:
---------------------
{context_str}
---------------------

**QUY TẮC PHẢN HỒI**:
- KHÔNG sử dụng kiến thức bên ngoài.
- Phải **trích dẫn trực tiếp** từ tài liệu khi trả lời.
- Sau phần trích dẫn, hãy **tóm tắt và giải thích ngắn gọn** dựa trên nội dung đó.

**Câu hỏi**: {query_str}

**Ví dụ minh hoạ**:
- Tài liệu liên quan: điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức: ...
- Câu trả lời mẫu: Theo điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức... Như vậy, bạn có thể chọn phương thức phù hợp với năng lực của mình. Chúc bạn thi tốt!




<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [168]:
query_engine_tool = QueryEngineTool.from_defaults(query_engine, name="retrieval_tool", description="use this tool to retrieve relevant documents from the database via user's query")
query_engine_tool.call(query)

ToolOutput(content='Điều 8. Xây dựng, thẩm định học liệu điện tử thuộc QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN có quy định: "Các ĐVQLMH có trách nhiệm thẩm định, đảm bảo chất lượng học liệu điện tử phục vụ DHTT, DHKH đối với các môn học mà đơn vị phụ trách. Trường có thể tổ chức thẩm định lại trong trường hợp cần thiết."\n\nNhư vậy, sau khi được thông qua bởi ĐVQLMH, học liệu điện tử vẫn có thể được trường tổ chức thẩm định lại nếu cần thiết.\n', tool_name='retrieval_tool', raw_input={'input': 'Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?'}, raw_output=Response(response='Điều 8. Xây dựng, thẩm định học liệu điện tử thuộc QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN có quy định: "Các ĐVQLMH có trách nhiệm thẩm định, đảm bảo chất lượng học liệu điện tử phục vụ DHTT, DHKH đối với các môn học mà đơn vị phụ tr

# Create a FunctionTool
A FunctionTool provides a simple way to wrap any Python function and make it available to an agent <br>
document: https://huggingface.co/learn/agents-course/en/unit2/llama-index/tools#creating-a-functiontool

In [21]:
from llama_index.core.tools import FunctionTool

def retrieve_database(query: str) -> str:
    """This function is used to retrieve relevant documents from the vector databse using the query.
        
        Arguments:
            query: str: The query string to search for in the database.
            
        Returns:
            str: The retrieved relevant documents formatted as a string.
    """
    
    retriever = index.as_retriever(similarity_top_k=10, vector_store_query_mode="hybrid", alpha=0.5)
    nodes = retriever.retrieve(query)
    context_str = "\n".join([f'{node.metadata['article']}, {node.metadata['document']}, {node.get_content()}' 
                            for id, node in enumerate(nodes)])
    return context_str


tool_retrieve = FunctionTool.from_defaults(
    fn=retrieve_database,
    name="retrieve_database",
    description="Tool to retrieve relevant documents from database using given query as input",
)

tool_retrieve.call(query)


ToolOutput(content='Điều 8. Xây dựng, thẩm định học liệu điện tử, QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 8. Xây dựng, thẩm định học liệu điện tử 1. Học liệu điện tử sử dụng trong DHTT và DHKH phải có nội dung phù hợp với đề cương môn học được phê duyệt (bao gồm chuẩn đầu ra môn học), bảo đảm tính khoa học, sư phạm và đáp ứng được nhu cầu tự học của sinh viên. 2. Học liệu điện tử được sử dụng từ nguồn sẵn có của Trường hoặc giảng viên chủ động xây dựng. 3. Khi xây dựng và sử dụng học liệu điện tử, giảng viên có trách nhiệm đảm bảo các quy định của pháp luật về Luật Sở hữu trí tuệ hiện hành. 4. Các ĐVQLMH có trách nhiệm thẩm định, đảm bảo chất lượng học liệu điện tử phục vụ DHTT, DHKH đối với các môn học mà đơn vị phụ trách. Trường có thể tổ chức thẩm định lại trong trường hợp cần thiết.\nĐiều 9. Sử dụng học liệu điện tử, QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC C

# Agent

LlamaIndex: https://www.llamaindex.ai/blog/introducing-agentworkflow-a-powerful-system-for-building-ai-agent-systems <br>
HuggingFace: https://huggingface.co/learn/agents-course/en/unit2/llama-index/agents

Streaming ouput: 
* https://docs.llamaindex.ai/en/stable/understanding/agent/streaming/
* [Async Programming in Python](https://docs.llamaindex.ai/en/stable/getting_started/async_python/#async-programming-in-python)

AgentStream is just one of many events that AgentWorkflow emits as it runs. The others are: <br>
* `AgentInput`: Captures the initial message that starts the agent's execution.
* `ToolCall`: Logs which tools were called and with what arguments.
* `ToolCallResult`: Displays the result of a tool call, including the tool name, arguments, and output.
* `AgentOutput`: Shows the final response from the agent.
* `AgentStream`: Optionally included to show the thought process or intermediate steps.

In [28]:
TEMPLATE_RESPONSE_ABUSE = "Câu hỏi của bạn có vẻ như là lạm dụng hệ thống của chúng tôi. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp."
TEMPLATE_RESPONSE_SOCIAL = "Xin chào! Tôi là chuyên gia tư vấn tuyển sinh UIT. Bạn có cần tôi hỗ trợ gì không?"
TEMPLATE_RESPONSE_OUT_OF_DOMAIN = "Câu hỏi của bạn không nằm trong phạm vi tuyển sinh của UIT. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp."
TEMPLATE_RESPONSE_NOT_VIETNAMESE = "Câu hỏi của bạn không phải là tiếng Việt. Bạn vui lòng đặt câu hỏi bằng tiếng Việt để nhận được trợ giúp. Your question is not in Vietnamese. Please ask questions in Vietnamese for support."
TEMPLATE_RESPONSE_NO_INFORMATION = "Không tìm thấy thông tin cho câu hỏi của bạn trong cơ sở dữ liệu. Vui lòng đặt lại câu hỏi hoặc liên hệ với phòng tư vấn tuyển sinh"

In [18]:
system_prompt_single_agent = f"""
Bạn là một chuyên gia tư vấn tuyển sinh của Trường Đại học Công nghệ Thông Tin - TP.HCM (UIT).
Bạn phải xử lý câu hỏi của người dùng theo **2 bước sau**:

**BƯỚC 1: PHÂN LOẠI CÂU HỎI**

Ở bước 1, bạn KHÔNG sử dụng tools để truy cập vào cơ sở dữ liệu ngày lập tức.
Dựa trên nội dung câu hỏi, bạn phải xác định câu hỏi thuộc **một trong 5 loại** sau:

1. **Câu xã giao**:  
   - Là những câu bằng tiếng Việt dùng để bắt đầu cuộc hội thoại, không chứa yêu cầu thông tin cụ thể.  
   
     *Ví dụ:*  
     - "Xin chào, bạn có thể làm gì?"  
     - "Chào bạn, tư vấn gì vậy?"
     
2. **in-domain**:  
   - Câu hỏi bằng tiếng Việt và liên quan đến:
      + Tuyển sinh (phương thức xét tuyển, điểm chuẩn, ngành học, hồ sơ, điều kiện nhập học,...)
      + Chương trình đào tạo (môn học, lộ trình học tập, chuẩn đầu ra, học liệu, phương pháp giảng dạy,...)
      + Cơ sở vật chất (thư viện, phòng máy, ký túc xá,...)
      + Học phí, học bổng, hỗ trợ tài chính
      + Hoạt động sinh viên, giảng viên, cán bộ, câu lạc bộ, sự kiện trong trường
      
      *Ví dụ:*
      - Điểm của khoá luận tính theo thang điểm mấy?
	  - Các đơn vị được phân công quản lý việc cấp văn bằng, chứng chỉ có phải có trách nhiệm cấp bản sao văn bằng, chứng chỉ từ sổ gốc theo quy định không? 
	  - Xét tuyển theo những phương thức nào?
	  - Có bao nhiêu chương trình học? 
	  - Nếu sinh viên chưa đạt học phần/chứng chỉ bắt buộc thì phải làm gì?

3. **out-of-domain**:  
   - Là câu hỏi **không tập trung vào UIT** hoặc **không liên quan đến các nội dung thuộc UIT**. Bao gồm:
     + Hỏi về **trường đại học khác**
     + So sánh UIT với trường khác
     + Hỏi về **ngành học không có tại UIT**
     + Hỏi về **chủ đề ngoài phạm vi quản lý của UIT**

     *Ví dụ:*
     - "Điểm chuẩn Đại học Bách Khoa 2024?"  
     - "UIT và BK trường nào giỏi công nghệ thông tin hơn?"  
     - "Trường nào đào tạo IT tốt nhất giữa các trường A, B, C?"

4. **prompt abuse**:  
   - Là câu hỏi mang tính xúc phạm, khiêu khích, spam hoặc làm dụng hệ thống với những cầu hỏi không phù hợp với phạm vị học đường

     *Ví dụ:*  
     - "@#!% UIT có dễ vào không?"  
     - "Hack điểm thi UIT được không?"

5. **not vietnamese**:  
   - Là câu hỏi **không được viết bằng tiếng Việt**

     *Ví dụ:*  
     - "What is the admission deadline?"  
     - "UITの入学基準は何ですか?"

**BƯỚC 2: TRẢ LỜI CÂU HỎI CHÍNH XÁC TỪNG CÂU CHỮ ĐÚNG THEO KỊCH BẢN MẪU (DỰA TRÊN LOẠI ĐÃ PHÂN LOẠI Ở BƯỚC 1)**

- Nếu câu hỏi là **câu xã giao bằng tiếng Việt**:
  → Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_SOCIAL}

- Nếu câu hỏi là **out-of-domain**:  
  → Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_OUT_OF_DOMAIN}

- Nếu câu hỏi là **prompt abuse**:  
  → Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_ABUSE}

- Nếu câu hỏi là **not vietnamese**:  
  → Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_NOT_VIETNAMESE}

- Nếu câu hỏi **không thuộc bất kỳ loại nào ở trên** (tức là **in-domain**), bạn phải sử dụng tool retrieve_database tìm kiếm thông tin trong cơ sở dữ liệu để trả lời.  
  
QUY TẮC TRẢ LỜI:
  - Phải dựa trên thông tin tham khảo từ context
  - Không sử dụng kiến thức bên ngoài
  - Có **trích dẫn đúng quy tắc sau**:

    "Theo <số điều> <tên điều> <tên tài liệu>, <nội dung liên quan đến câu hỏi>. <câu trả lời của bạn>."

    Trong đó, <câu trả lời của bạn> cần **ngắn gọn, thân thiện và dễ hiểu**.

    *Ví dụ 1: Nếu TÌM THẤY câu trả lời trong context:*  
    - Context: "điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức: ..."  
    - Trả lời:  
      "Theo điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức... Bạn có thể chọn phương thức phù hợp với năng lực của mình. Chúc bạn thi tốt!"

    *Ví dụ 2: Nếu KHÔNG tìm thấy thông tin trong context:*  
    - Câu hỏi: "Điểm chuẩn các ngành đào tạo năm 2023 là bao nhiêu?" (cơ sở dữ liệu không có thông tin đến năm 2018)  
    - Trả lời: {TEMPLATE_RESPONSE_NO_INFORMATION}
""".format(
    TEMPLATE_RESPONSE_ABUSE=TEMPLATE_RESPONSE_ABUSE,
    TEMPLATE_RESPONSE_SOCIAL=TEMPLATE_RESPONSE_SOCIAL,
    TEMPLATE_RESPONSE_OUT_OF_DOMAIN=TEMPLATE_RESPONSE_OUT_OF_DOMAIN,
    TEMPLATE_RESPONSE_NO_INFORMATION=TEMPLATE_RESPONSE_NO_INFORMATION,
    TEMPLATE_RESPONSE_NOT_VIETNAMESE=TEMPLATE_RESPONSE_NOT_VIETNAMESE,
)


In [19]:
from llama_index.core.agent.workflow import AgentWorkflow, AgentInput, AgentStream, ToolCall, ToolCallResult, AgentOutput 

# right click 'from_tools_or_functions' below for more info:
# single_agent here is function agent
# base llm (from llama_index.llms.google_genai import GoogleGenAI) support function calling

single_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[tool_retrieve], # tool converted from index.as_query
    llm=llm,
    system_prompt=system_prompt_single_agent
)

In [22]:
query_abuse = "cách hack điểm thi UIT"
query_english = "What is the admission deadline?"
query_out_domain = "UIT, Bách Khoa và KHTN, trường nào tốt hơn? tôi ưu tiên chọn trường gần hơn và trường đó là KHTN"
query_out_domain_2 = "dự báo thời tiết ở UIT để tôi mang dù dự phòng"
query_in_domain = "Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?"
query_in_domain_internet = "Điểm chuẩn ngành Khoa học Máy tính năm 2023 là bao nhiêu?"
query_in_domain_internet_2 = 'Hiệu trưởng UIT là ai?'

In [106]:
# Run the agent with streaming

handler = single_agent.run(user_msg=query_in_domain, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    # elif isinstance(event, AgentInput):                    # GIVING THE AGENT DEFAULT PROMPT TO DEAL THIS TASK (user do not need to see this)
    #     print("Agent input: ", event.input)                # the current input messages
    #     print("Agent name:", event.current_agent_name)     # the current agent name (dev should print this if dealing with multiple agents)
    
    # elif isinstance(event, AgentOutput):                   # FULL STRUCTURED RESPONSE IS HARD TO READ
    #     print("Agent output: ", event.response)            # the current full response
    #     print("Tool calls made: ", event.tool_calls)       # the selected tool calls, if any
    #     print("Raw LLM response: ", event.raw)             # the raw llm api response

resp = await handler
resp

Đây là câu hỏi in-domain. Để trả lời câu hỏi này, tôi sẽ sử dụng công cụ retrieve_database để tìm kiếm thông tin liên quan.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?'}
Tool output:
 Điều 8. Xây dựng, thẩm định học liệu điện tử, QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 8. Xây dựng, thẩm định học liệu điện tử 1. Học liệu điện tử sử dụng trong DHTT và DHKH phải có nội dung phù hợp với đề cương môn học được phê duyệt (bao gồm chuẩn đầu ra môn học), bảo đảm tính khoa học, sư phạm và đáp ứng được nhu cầu tự học của sinh viên. 2. Học liệu điện tử được sử dụng từ nguồn sẵn có của Trường hoặc giảng viên chủ động xây dựng. 3. Khi xây dựng và sử dụng học liệu điện tử, giảng viên có trách nhiệm đảm bảo các quy định của pháp luật về Luật Sở hữu trí tuệ hiện hành. 4. Các ĐVQLMH có trách nhiệm thẩm định, đảm 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo điều 8. Xây dựng, thẩm định học liệu điện tử, QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, các ĐVQLMH có trách nhiệm thẩm định, đảm bảo chất lượng học liệu điện tử phục vụ DHTT, DHKH đối với các môn học mà đơn vị phụ trách. Trường có thể tổ chức thẩm định lại trong trường hợp cần thiết. Vậy nên, học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa nếu trường thấy cần thiết bạn nhé.\n')]), tool_calls=[ToolCallResult(tool_name='retrieve_database', tool_kwargs={'query': 'Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?'}, tool_id='retrieve_database', tool_output=ToolOutput(content='Điều 8. Xây dựng, thẩm định học liệu điện tử, QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾ

In [37]:
handler = single_agent.run(user_msg=query_abuse, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process


resp = await handler
resp

Câu hỏi của bạn có vẻ như là lạm dụng hệ thống của chúng tôi. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Câu hỏi của bạn có vẻ như là lạm dụng hệ thống của chúng tôi. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' đặt câu hỏi khác để nhận được trợ giúp.\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 31, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'

In [38]:
handler = single_agent.run(user_msg=query_english, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process


resp = await handler
resp

Câu hỏi của bạn không phải là tiếng Việt. Bạn vui lòng đặt câu hỏi bằng tiếng Việt để nhận được trợ giúp. Your question is not in Vietnamese. Please ask questions in Vietnamese for support.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Câu hỏi của bạn không phải là tiếng Việt. Bạn vui lòng đặt câu hỏi bằng tiếng Việt để nhận được trợ giúp. Your question is not in Vietnamese. Please ask questions in Vietnamese for support.\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' Vietnamese for support.\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 41, 'candidates

In [39]:
handler = single_agent.run(user_msg=query_out_domain, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process


resp = await handler
resp

Câu hỏi của bạn không nằm trong phạm vi tuyển sinh của UIT. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Câu hỏi của bạn không nằm trong phạm vi tuyển sinh của UIT. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' đặt câu hỏi khác để nhận được trợ giúp.\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 28, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,

In [42]:
handler = single_agent.run(user_msg=query_in_domain_internet, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process


resp = await handler
resp

Đây là câu hỏi in-domain, tôi sẽ sử dụng tool để tìm kiếm thông tin.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'Điểm chuẩn ngành Khoa học Máy tính năm 2023 là bao nhiêu?'}
Tool output:
 Điều 16. Chấm thi, nhập điểm và công bố điểm, QUY ĐỊNH Tổ chức thi các môn học hệ đại học chính quy của Trường Đại học Công nghệ Thông tin, Điều 16. Chấm thi, nhập điểm và công bố điểm 1. ĐVQLMH của môn học có trách nhiệm tổ chức công tác chấm thi cho môn học mà mình quản lý. Khoa chịu trách nhiệm chính về sự chính xác, công bằng và các khía cạnh chuyên môn khác của việc chấm thi. 2. Đối với kỳ thi cuối học kỳ, các môn học có nhiều hơn 01 cán bộ giảng dạy phải có cán bộ chấm thi cho một lớp học khác với cán bộ giảng dạy của lớp học đó (chấm chéo). 3. Qui trình nhập điểm Bước 1: Nhận bài thi/dữ liệu về bài thi - Sau ngày thi, cán bộ chấm thi nhận túi bài thi/dữ liệu về bài thi tại đơn vị tổ chức thi. Bước 2: Cập nhật phân công chấm thi - Đơn vị quản lý môn học gửi danh sách phân 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Không tìm thấy thông tin cho câu hỏi của bạn trong cơ sở dữ liệu. Vui lòng đặt lại câu hỏi hoặc liên hệ với phòng tư vấn tuyển sinh\n')]), tool_calls=[ToolCallResult(tool_name='retrieve_database', tool_kwargs={'query': 'Điểm chuẩn ngành Khoa học Máy tính năm 2023 là bao nhiêu?'}, tool_id='retrieve_database', tool_output=ToolOutput(content='Điều 16. Chấm thi, nhập điểm và công bố điểm, QUY ĐỊNH Tổ chức thi các môn học hệ đại học chính quy của Trường Đại học Công nghệ Thông tin, Điều 16. Chấm thi, nhập điểm và công bố điểm 1. ĐVQLMH của môn học có trách nhiệm tổ chức công tác chấm thi cho môn học mà mình quản lý. Khoa chịu trách nhiệm chính về sự chính xác, công bằng và các khía cạnh chuyên môn khác của việc chấm thi. 2. Đối với kỳ thi cuối học kỳ, các môn học có nhiều hơn 01 cán bộ giảng dạy phải có cán bộ chấm thi cho một lớp học k

In [53]:
handler = single_agent.run(user_msg=query_in_domain_internet_2, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process


resp = await handler
resp

Đây là câu hỏi in-domain. Tôi sẽ sử dụng tool để tìm kiếm thông tin.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'Hiệu trưởng UIT là ai?'}
Tool output:
 Điều 2. Phân cấp quản lý công tác giáo trình, QUI ĐỊNH VỀ CÔNG TÁC GIÁO TRÌNH, Điều 2. Phân cấp quản lý công tác giáo trình 2.1. Hiệu trưởng ra quyết định thành lập Ban Điều hành Công tác giáo trình (gọi tắt là Ban Điều hành CTGT), thành phần gồm có: Hiệu trưởng/Phó Hiệu trưởng phụ trách đào tạo làm Trưởng ban, Trưởng/Phó phòng đào tạo làm ủy viên thường trực kiêm thư ký, cán bộ – viên chức khác làm ủy viên. 2.2. Trưởng Ban Điều hành CTGT chịu trách nhiệm trước Hiệu trưởng về tổ chức hoạt động của mảng công tác giáo trình và trước Chủ nhiệm CTGT ĐHQG về chuyên môn. 2.3. Chức năng của Ban Điều hành CTGT cụ thể như sau: • Tổ chức xây dựng kế hoạch biên soạn giáo trình căn cứ theo chương trình giáo dục, kế hoạch giảng dạy và học tập đối với 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Thông tin về hiệu trưởng UIT không được tìm thấy trong cơ sở dữ liệu. Vui lòng đặt lại câu hỏi hoặc liên hệ với phòng tư vấn tuyển sinh.\n')]), tool_calls=[ToolCallResult(tool_name='retrieve_database', tool_kwargs={'query': 'Hiệu trưởng UIT là ai?'}, tool_id='retrieve_database', tool_output=ToolOutput(content='Điều 2. Phân cấp quản lý công tác giáo trình, QUI ĐỊNH VỀ CÔNG TÁC GIÁO TRÌNH, Điều 2. Phân cấp quản lý công tác giáo trình 2.1. Hiệu trưởng ra quyết định thành lập Ban Điều hành Công tác giáo trình (gọi tắt là Ban Điều hành CTGT), thành phần gồm có: Hiệu trưởng/Phó Hiệu trưởng phụ trách đào tạo làm Trưởng ban, Trưởng/Phó phòng đào tạo làm ủy viên thường trực kiêm thư ký, cán bộ – viên chức khác làm ủy viên. 2.2. Trưởng Ban Điều hành CTGT chịu trách nhiệm trước Hiệu trưởng về tổ chức ho

In [47]:
single_agent_prompt_dict =  single_agent.get_prompts()
display_prompt_dict(single_agent_prompt_dict)

**Prompt Key**: handoff_prompt<br>**Text:** <br>

Useful for handing off to another agent.
If you are currently not equipped to handle the user's request, or another agent is better suited to handle the request, please hand off to the appropriate agent.

Currently available agents:
{agent_info}



<br><br>

**Prompt Key**: handoff_output_prompt<br>**Text:** <br>

Agent {to_agent} is now handling the request due to the following reason: {reason}.
Please continue with the current request.


<br><br>

**Prompt Key**: state_prompt<br>**Text:** <br>

Current state:
{state}

Current message:
{msg}



<br><br>

# Multi-Agent

HuggingFace: 
* https://huggingface.co/learn/agents-course/en/unit2/llama-index/agents#creating-multi-agent-systems 

LlamaIndex: 
* https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/ 
* https://docs.llamaindex.ai/en/stable/understanding/agent/

In this section, we use only: *ReAct Agent* <br>
Possible error when integrating tools in **FunctionAgent** with **Gemini** (from llama_index.llms.gemini import Gemini)

`WorkflowRuntimeError: Error in step 'run_agent_step': LLM must be a FunctionCallingLLM`

1. *ReActAgent: This agent type is designed to work with any LLM that can generate text based on prompts. It doesn't directly invoke tools but instead relies on the LLM's ability to interpret prompts that suggest tool usage. This approach is more flexible and can be used with a wider range of LLMs, including those that do not have built-in support for function calling. The downside is that it might be less precise, as it depends on the LLM's understanding of the prompt to correctly use the tools.*

2. *FunctionAgent: This agent type is specifically designed for LLMs that support direct function calling through their API, such as OpenAI's models. This allows for a more structured and reliable interaction with tools, as the LLM can directly execute functions with specified parameters. This capability is particularly useful for complex workflows where precise control over tool usage is necessary.*

**The main advantage of the ReAct agent over a Function Calling agent is that it can work with any LLM regardless of whether it supports function calling.**

Connecting our LLM to the Web via [tavily](https://tavily.com/) <br>
we will have **1000 credits/month**

In [17]:
# !pip install tavily-python (0.5.1)
# !pip show tavily-python

In [23]:
from tavily import AsyncTavilyClient

async def search_web(query: str) -> str:
    """
    Search information on internet
    Args:
        query (str): The search query.
    Returns:
        str: The search result.
    """
    
    print(f'searching web via query: {query}')
    
    client = AsyncTavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    return str(await client.search(query, max_results=5,))

In [30]:
result = await search_web(query=query_in_domain_internet_2)
print(result)

searching web via query: Hiệu trưởng UIT là ai?
{'query': 'Hiệu trưởng UIT là ai?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Tái bổ nhiệm TS. Nguyễn Hoàng Tú Anh làm Hiệu trưởng UIT', 'url': 'https://doanhnhansaigon.vn/tai-bo-nhiem-ts-nguyen-hoang-tu-anh-lam-hieu-truong-uit-206483.html', 'content': 'Chiều 12/4/2022, PGS.TS. Vũ Hải Quân - Ủy viên Ban chấp hành Trung ương Đảng, Bí thư Đảng ủy, Giám đốc Đại học (ĐH) Quốc gia TP.HCM đã trao quyết định tái bổ nhiệm Hiệu trưởng ĐH Công nghệ Thông tin (UIT) cho TS. Nguyễn Hoàng Tú Anh. Đây là nhiệm kỳ thứ hai của TS. Nguyễn Hoàng Tú Anh tại UIT.', 'score': 0.83225065, 'raw_content': None}, {'title': 'Trường đại học đầu tiên tại Việt Nam sở hữu máy chủ AI 10 tỷ đồng', 'url': 'https://metazones.vn/truong-dai-hoc-dau-tien-tai-viet-nam-so-huu-may-chu-ai-10-ty-dong', 'content': 'Máy đóng gói mật độ cao các máy tính với hiệu suất xử lý AI lên đến 5 petaFLOPS trong khung máy 6U, thay thế tháp cơ sở hạ tầng ch

In [24]:
from llama_index.core.tools import FunctionTool

tool_search = FunctionTool.from_defaults(
    fn=search_web,
    name="search_web",
    description="search information on internet",
)

In [25]:
# system_prompt_filter_agent = """
# Bạn là một chuyên gia có nhiệm vụ phân loại câu hỏi dựa trên nội dung của chúng.
# Công việc của bạn bao gồm **2 bước**: phân loại và phản hồi theo đúng kịch bản tương ứng.

# **BƯỚC 1: PHÂN LOẠI CÂU HỎI**

# Ở bước 1, sử dụng khả năng đọc hiểu của bạn và dựa trên nội dung câu hỏi, hãy xác định câu hỏi thuộc **một trong 5 loại** sau:

# 1. **Câu xã giao**:  
#    - Là những câu bằng tiếng Việt dùng để bắt đầu cuộc hội thoại, không chứa yêu cầu thông tin cụ thể.  
   
#      *Ví dụ:*  
#      - "Xin chào, bạn có thể làm gì?"  
#      - "Chào bạn, tư vấn gì vậy?"
     
# 2. **in-domain**:  
#    - Câu hỏi bằng tiếng Việt và liên quan trực tiếp đến trường UIT (có thể không nhắc đến tên trường những vẫn có thể liên quan):
#       + Tuyển sinh (phương thức xét tuyển, điểm chuẩn, ngành học, hồ sơ, điều kiện nhập học,...)
#       + Chương trình đào tạo (môn học, lộ trình học tập, chuẩn đầu ra, học liệu, phương pháp giảng dạy,...)
#       + Cơ sở vật chất (thư viện, phòng máy, ký túc xá,...)
#       + Học phí, học bổng, hỗ trợ tài chính
#       + Hoạt động sinh viên, giảng viên, cán bộ, câu lạc bộ, sự kiện trong trường
      
#       *Ví dụ:*
#       - Điểm của khoá luận tính theo thang điểm mấy?
# 	  - Các đơn vị được phân công quản lý việc cấp văn bằng, chứng chỉ có phải có trách nhiệm cấp bản sao văn bằng, chứng chỉ từ sổ gốc theo quy định không? 
# 	  - Xét tuyển theo những phương thức nào?
# 	  - Có những chương trình học nào? 
# 	  - Nếu sinh viên chưa đạt học phần/chứng chỉ bắt buộc thì phải làm gì?
# 	  - Ai là hiệu phó của UIT hiện tại?
#       - Trường UIT ở đâu?

# 3. **out-of-domain**:  
#    - Là câu hỏi **không tập trung vào UIT** hoặc **không liên quan đến UIT**. Bao gồm:
#      + Hỏi về **trường đại học khác**
#      + So sánh UIT với trường khác
#      + Hỏi về **ngành học không có tại UIT**
#      + Hỏi về **chủ đề ngoài phạm vi quản lý của UIT**

#      *Ví dụ:*
#      - "Điểm chuẩn Đại học Bách Khoa 2024?"  
#      - "UIT và BK trường nào giỏi công nghệ thông tin hơn?"  
#      - "Trường nào đào tạo IT tốt nhất giữa các trường A, B, C?"

# 4. **prompt abuse**:  
#    - Là câu hỏi mang tính xúc phạm, khiêu khích, spam hoặc làm dụng hệ thống với những cầu hỏi không phù hợp với phạm vị học đường

#      *Ví dụ:*  
#      - "@#!% UIT có dễ vào không?"  
#      - "Hack điểm thi UIT được không?"

# 5. **not vietnamese**:  
#    - Là câu hỏi **không được viết bằng tiếng Việt**

#      *Ví dụ:*  
#      - "What is the admission deadline?"  
#      - "UITの入学基準は何ですか?"

# **BƯỚC 2: HÀNG ĐỘNG TƯƠNG ỨNG VỚI TỪNG CÂU HỎI (DỰA TRÊN LOẠI ĐÃ PHÂN LOẠI Ở BƯỚC 1)**:

# 	Bạn chỉ thực hiện duy nhất 1 trong 2 việc: chuyên giao hoặc trả lời câu hỏi rồi kết thúc

# 	Trường hợp 1: Nếu câu hỏi là **in-domain**:
 
# 	Chuyển giao **CÂU HỎI HIỆN TẠI** cho Answer_Agent (Answer_Agent sẽ đảm nhiệm tốt hơn vai trò trả lời câu hỏi)

# 	Trường hợp 2: Nếu câu hỏi thuộc **một trong 4 loại** sau, bạn phải TRẢ LỜI CHÍNH XÁC TỪNG CÂU CHỮ ĐÚNG THEO KỊCH BẢN MẪU. 

# 	- Nếu câu hỏi là **câu xã giao bằng tiếng Việt**:
# 	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_SOCIAL}

# 	- Nếu câu hỏi là **out-of-domain**:  
# 	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_OUT_OF_DOMAIN}

# 	- Nếu câu hỏi là **prompt abuse**:  
# 	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_ABUSE}

# 	- Nếu câu hỏi là **not vietnamese**:  
# 	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_NOT_VIETNAMESE}
 
# 	Sau khi đã phản hồi chích xác theo kịch bản, **KHÔNG ĐƯỢC** chuyển giao cho Answer_Agent và bạn **PHẢI** KẾT THÚC CUỘC TRÒ CHUYỆN NGAY LẬP TỨC để bảo vệ hệ thống.

	
# """.format(
#     TEMPLATE_RESPONSE_ABUSE=TEMPLATE_RESPONSE_ABUSE,
#     TEMPLATE_RESPONSE_SOCIAL=TEMPLATE_RESPONSE_SOCIAL,
#     TEMPLATE_RESPONSE_OUT_OF_DOMAIN=TEMPLATE_RESPONSE_OUT_OF_DOMAIN,
#     TEMPLATE_RESPONSE_NO_INFORMATION=TEMPLATE_RESPONSE_NO_INFORMATION,
#     TEMPLATE_RESPONSE_NOT_VIETNAMESE=TEMPLATE_RESPONSE_NOT_VIETNAMESE,
# )


# system_prompt_answer_agent = """
# Bạn là một chuyên gia tư vấn tuyển sinh của Trường Đại học Công nghệ Thông Tin (UIT).
# Nhiệm vụ của bạn là trả lời các câu hỏi của người dùng dựa trên thông tin thu thập được từ hai công cụ: `retrieve_database` và `search_web`.

# **Nguyên tắc sử dụng công cụ:**
# 1. Luôn ưu tiên sử dụng công cụ `retrieve_database` trước.
# 2. Nếu không tìm thấy thông tin, hoặc thông tin chưa đủ rõ ràng để đưa ra câu trả lời chính xác, bạn **PHẢI** tiếp tục sử dụng `search_web` để tra cứu thêm.
# 3. Khi công cụ `retrieve_database` chưa trả về thông tin liên quan, bạn ít nhất phải sủ dụng công cụ `search_web` 1 lần nữa để tìm câu trả lời tốt hơn.

# **Cách trả lời:**
# - Nếu tìm thấy thông tin phù hợp, hãy trả lời theo cấu trúc sau:

#     Theo <số điều> <tên điều>, <nội dung tài liệu liên quan đến câu hỏi>. <Câu trả lời ngắn gọn, thân thiện của bạn>.

#     **Ví dụ:**
#     - Context: "Điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức: ..."
#     - Trả lời: "Theo điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức... Như vậy, bạn có thể chọn phương thức phù hợp với năng lực của mình. Chúc bạn thi tốt!"

# - Nếu đã sử dụng cả hai công cụ hỗ trợ mà vẫn **không tìm thấy thông tin** liên quan hoặc thông tin **chưa đủ rõ** để đưa ra câu trả lời, bạn **PHẢI** trả lời theo mẫu sau:
#     {TEMPLATE_RESPONSE_NO_INFORMATION}
# """.format(
#     TEMPLATE_RESPONSE_NO_INFORMATION=TEMPLATE_RESPONSE_NO_INFORMATION
# )


In [49]:
system_prompt_filter_agent = """
Bạn là một chuyên gia phân loại câu hỏi dựa trên nội dung của chúng.
Công việc của bạn bao gồm **2 bước**: phân loại và phản hồi theo đúng kịch bản tương ứng.

**BƯỚC 1: PHÂN LOẠI CÂU HỎI**

Ở bước 1, sử dụng khả năng đọc hiểu của bạn và dựa trên nội dung câu hỏi, hãy xác định câu hỏi thuộc **một trong 5 loại** sau:

1. **Câu xã giao**:  
   - Là những câu bằng tiếng Việt dùng để bắt đầu cuộc hội thoại, không chứa yêu cầu thông tin cụ thể.  
   
     *Ví dụ:*  
     - "Xin chào, bạn có thể làm gì?"  
     - "Chào bạn, tư vấn gì vậy?"
     
2. **in-domain**:  
   - Câu hỏi bằng tiếng Việt và liên quan trực tiếp đến trường UIT (có thể không nhắc đến tên trường những vẫn có thể liên quan):
      + Tuyển sinh (phương thức xét tuyển, điểm chuẩn, ngành học, hồ sơ, điều kiện nhập học,...)
      + Chương trình đào tạo (môn học, lộ trình học tập, chuẩn đầu ra, học liệu, phương pháp giảng dạy,...)
      + Cơ sở vật chất (thư viện, phòng máy, ký túc xá,...)
      + Học phí, học bổng, hỗ trợ tài chính
      + Hoạt động sinh viên, giảng viên, cán bộ, câu lạc bộ, sự kiện trong trường
      
      *Ví dụ:*
      - Điểm của khoá luận tính theo thang điểm mấy?
	  - Các đơn vị được phân công quản lý việc cấp văn bằng, chứng chỉ có phải có trách nhiệm cấp bản sao văn bằng, chứng chỉ từ sổ gốc theo quy định không? 
	  - Xét tuyển theo những phương thức nào?
	  - Có những chương trình học nào? 
	  - Nếu sinh viên chưa đạt học phần/chứng chỉ bắt buộc thì phải làm gì?
	  - Ai là hiệu phó của UIT hiện tại?
      - Trường UIT ở đâu?

3. **out-of-domain**:  
   - Là câu hỏi **không tập trung vào UIT** hoặc **không liên quan đến UIT**. Bao gồm:
     + Hỏi về **trường đại học khác**
     + So sánh UIT với trường khác
     + Hỏi về **ngành học không có tại UIT**
     + Hỏi về **chủ đề ngoài phạm vi quản lý của UIT**

     *Ví dụ:*
     - "Điểm chuẩn Đại học Bách Khoa 2024?"  
     - "UIT và BK trường nào giỏi công nghệ thông tin hơn?"  
     - "Trường nào đào tạo IT tốt nhất giữa các trường A, B, C?"

4. **prompt abuse**:  
   - Là câu hỏi mang tính xúc phạm, khiêu khích, spam hoặc làm dụng hệ thống với những cầu hỏi không phù hợp với phạm vị học đường

     *Ví dụ:*  
     - "@#!% UIT có dễ vào không?"  
     - "Hack điểm thi UIT được không?"

5. **not vietnamese**:  
   - Là câu hỏi **không được viết bằng tiếng Việt**

     *Ví dụ:*  
     - "What is the admission deadline?"  
     - "UITの入学基準は何ですか?"

**BƯỚC 2: HÀNG ĐỘNG TƯƠNG ỨNG VỚI TỪNG CÂU HỎI (DỰA TRÊN LOẠI ĐÃ PHÂN LOẠI Ở BƯỚC 1)**:

	Bạn chỉ thực hiện duy nhất 1 trong 2 việc: chuyên giao hoặc trả lời câu hỏi rồi kết thúc

	Trường hợp 1: Nếu câu hỏi là **in-domain**:
 
	Chuyển giao **CÂU HỎI HIỆN TẠI** cho Retrieve_Agent (Retrieve_Agent sẽ đảm nhiệm tốt hơn vai trò trả lời câu hỏi)

	Trường hợp 2: Nếu câu hỏi thuộc **một trong 4 loại** sau, bạn phải TRẢ LỜI CHÍNH XÁC TỪNG CÂU CHỮ ĐÚNG THEO KỊCH BẢN MẪU. 

	- Nếu câu hỏi là **câu xã giao bằng tiếng Việt**:
	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_SOCIAL}

	- Nếu câu hỏi là **out-of-domain**:  
	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_OUT_OF_DOMAIN}

	- Nếu câu hỏi là **prompt abuse**:  
	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_ABUSE}

	- Nếu câu hỏi là **not vietnamese**:  
	→ Trả lời theo mẫu sau: {TEMPLATE_RESPONSE_NOT_VIETNAMESE}
 
	Sau khi đã phản hồi chích xác theo kịch bản, **KHÔNG ĐƯỢC** chuyển giao cho Retrieve_Agent và bạn **PHẢI** KẾT THÚC CUỘC TRÒ CHUYỆN NGAY LẬP TỨC để bảo vệ hệ thống.

""".format(
    TEMPLATE_RESPONSE_ABUSE=TEMPLATE_RESPONSE_ABUSE,
    TEMPLATE_RESPONSE_SOCIAL=TEMPLATE_RESPONSE_SOCIAL,
    TEMPLATE_RESPONSE_OUT_OF_DOMAIN=TEMPLATE_RESPONSE_OUT_OF_DOMAIN,
    TEMPLATE_RESPONSE_NO_INFORMATION=TEMPLATE_RESPONSE_NO_INFORMATION,
    TEMPLATE_RESPONSE_NOT_VIETNAMESE=TEMPLATE_RESPONSE_NOT_VIETNAMESE,
)


system_prompt_retrieve_agent = """
Bạn là một chuyên gia tư vấn tuyển sinh của Trường Đại học Công nghệ Thông Tin (UIT).
Nhiệm vụ của bạn là trả lời các câu hỏi của người dùng dựa trên **thông tin thu thập từ cơ sở dữ liệu nội bộ** thông qua công cụ: `retrieve_database`.

**LUẬT BẮT BUỘC**:
Bạn **KHÔNG BAO GIỜ ĐƯỢC PHÉP** đưa ra bất kỳ kết luận, hay chuyển giao tác vụ khi **chưa gọi công cụ `retrieve_database`**.

- Bạn **PHẢI** luôn luôn gọi công cụ `retrieve_database` với câu hỏi của người dùng để kiểm tra dữ liệu nội bộ trước tiên.
- Chỉ sau khi **đã gọi và đánh giá kết quả trả về**, bạn mới được đưa ra quyết định kế tiếp.

**Hướng dẫn đánh giá kết quả từ công cụ `retrieve_database`**:
- Nếu kết quả chứa thông tin rõ ràng, có liên quan đến câu hỏi, bạn **phải tạo câu trả lời dựa trên nội dung đó**.
- Nếu kết quả **không liên quan** đến câu hỏi, bạn **phải chuyển giao câu hỏi cho `Search_Agent` để tiếp tục xử lý**.

Bạn TUYỆT ĐỐI KHÔNG:
- Không tự ý nói "không tìm thấy", "xin lỗi", hay "tôi không biết" nếu chưa gọi `retrieve_database`.
- Không tự ý kết thúc cuộc trò chuyện hoặc trả lời chung chung nếu không có dữ kiện từ cơ sở dữ liệu.

---

**Cách trình bày câu trả lời nếu tìm được thông tin**:

Theo <số điều> <tên điều>, <nội dung tài liệu liên quan đến câu hỏi>. <Một câu kết thân thiện, khuyến khích hoặc động viên>.

**Ví dụ**:
- Context: "Điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức: ..."
- Trả lời: "Theo điều 9. Phương thức xét tuyển thuộc QUY CHẾ TUYỂN SINH ĐẠI HỌC, UIT xét tuyển theo 4 phương thức... Bạn có thể cân nhắc chọn phương thức phù hợp với năng lực của mình. Chúc bạn học tốt!"

---

**Nếu không tìm thấy thông tin (sau khi đã tra cứu)**:
- Bạn **phải chuyển câu hỏi cho `Search_Agent`** bằng cách gọi công cụ phù hợp và **KHÔNG TỰ TRẢ LỜI** nữa.

""" 


system_prompt_search_agent = """
Bạn là một chuyên gia tư vấn tuyển sinh của Trường Đại học Công nghệ Thông Tin (UIT).
Nhiệm vụ của bạn là **chỉ sử dụng công cụ `search_web` để tra cứu thông tin từ internet về Trường UIT**, nhằm hỗ trợ người dùng khi thông tin nội bộ không đủ.

---

**LUẬT BẮT BUỘC**:

1. Bạn **PHẢI gọi công cụ `search_web` trước khi đưa ra bất kỳ câu trả lời nào**. Không được phép trả lời nếu chưa tìm kiếm.
2. Khi gọi công cụ `search_web`, bạn **KHÔNG được dùng nguyên câu hỏi của người dùng nếu nó quá chung chung**. Bạn **PHẢI rõ ràng rằng chỉ tìm thông tin liên quan đến UIT** (ví dụ: "Điểm chuyển ngành Khoa học dữ liệu tại UIT", không phải "điểm chuyển ngành Khoa học dữ liệu").
3. Nếu không tìm được thông tin rõ ràng, hoặc kết quả không liên quan UIT, bạn **PHẢI** trả lời theo mẫu từ chối chính thức bên dưới.
4. **Chỉ sử dụng nguồn đáng tin cậy**: Các trang chính thức như `uit.edu.vn`, `tuyensinh.uit.edu.vn`, `vnexpress.net`, `tuoitre.vn`,... hoặc các báo chí chính thống.
5. **BẮT BUỘC** phải kèm theo **đường dẫn URL cụ thể** dẫn đến trang web chứa thông tin khi trích dẫn. Không được dẫn nguồn mơ hồ hoặc không có link.
6. Không bao giờ được dùng các nguồn không đáng tin cậy như blog cá nhân, diễn đàn, mạng xã hội, hoặc các trang không chính thức.

---

**CÁCH TRẢ LỜI KHI TÌM ĐƯỢC THÔNG TIN**:

Trả lời theo định dạng sau:

**Theo <nguồn thông tin/trang web> (<URL>), <trích dẫn nội dung phù hợp>. <Câu tóm tắt thân thiện, rõ ràng của bạn>.**

**Ví dụ**:
- Người dùng hỏi: Trường ĐH Công nghệ Thông tin có phải đại học công lập không?
- search_web trả về từ tuyensinh.uit.edu.vn: "Trường ĐH Công nghệ Thông tin là một trong 6 trường thành viên của ĐHQG-HCM"
- Trả lời: "Theo tuyensinh.uit.edu.vn (https://tuyensinh.uit.edu.vn/gioi-thieu), Trường ĐH Công nghệ Thông tin là một trong 6 trường đại học thành viên của ĐHQG-HCM. Vậy nên, đây là một trường đại học công lập bạn nhé!"

---

**NẾU KHÔNG TÌM THẤY THÔNG TIN LIÊN QUAN**, hoặc nếu **thông tin còn mơ hồ**, bạn phải trả lời chính xác theo mẫu:

{TEMPLATE_RESPONSE_NO_INFORMATION}

---

Lưu ý:
- Câu trả lời phải mang phong cách học thuật, rõ ràng, tích cực và thân thiện.
- KHÔNG tự suy đoán hoặc đưa ra ý kiến cá nhân.
- KHÔNG nói mơ hồ kiểu "có thể", "hình như", "theo tôi biết"...

""".format(TEMPLATE_RESPONSE_NO_INFORMATION=TEMPLATE_RESPONSE_NO_INFORMATION)


In [50]:
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent, FunctionAgent, ToolCallResult, AgentStream, AgentInput


filter_agent = FunctionAgent(
    name="Filter_Agent",
    description="You are a query classifier. Your task is take action correspond to each user queries",
    llm=llm,
    system_prompt=system_prompt_filter_agent,
    tools=[],
    can_handoff_to=["Retrieve_Agent"]
)


retrieve_agent = FunctionAgent(
    name="Retrieve_Agent",
    description="You are consultant and responsible for answering user questions by using retrieve database",
    llm=llm,
    system_prompt=system_prompt_retrieve_agent,
    tools=[tool_retrieve],
    can_handoff_to=['Search_Agent']
)

search_agent = FunctionAgent(
	name="Search_Agent",
	description="You are consultant and responsible for answering user questions by searching web for external information",
	llm=llm,
	system_prompt=system_prompt_search_agent,
	tools=[tool_search],
	can_handoff_to=[]
)


multi_agents = AgentWorkflow(
    agents=[filter_agent, retrieve_agent, search_agent],
    root_agent="Filter_Agent",
)


In [51]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg=query_abuse, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)       # the current agent name


resp = await handler
resp

Agent name: Filter_Agent
Câu hỏi của bạn có vẻ như là lạm dụng hệ thống của chúng tôi. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Câu hỏi của bạn có vẻ như là lạm dụng hệ thống của chúng tôi. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' đặt câu hỏi khác để nhận được trợ giúp.\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 31, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'

In [52]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg=query_out_domain, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)       # the current agent name


resp = await handler
resp

Agent name: Filter_Agent
Câu hỏi của bạn không nằm trong phạm vi tuyển sinh của UIT. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Câu hỏi của bạn không nằm trong phạm vi tuyển sinh của UIT. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' khác để nhận được trợ giúp.\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 28, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_coun

In [53]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg=query_english, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)       # the current agent name


resp = await handler
resp

Agent name: Filter_Agent
Câu hỏi của bạn không phải là tiếng Việt. Bạn vui lòng đặt câu hỏi bằng tiếng Việt để nhận được trợ giúp. Your question is not in Vietnamese. Please ask questions in Vietnamese for support.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Câu hỏi của bạn không phải là tiếng Việt. Bạn vui lòng đặt câu hỏi bằng tiếng Việt để nhận được trợ giúp. Your question is not in Vietnamese. Please ask questions in Vietnamese for support.\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' Vietnamese for support.\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 41, 'candidates

In [54]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg=query_in_domain, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)       # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'to_agent': 'Retrieve_Agent', 'reason': 'Câu hỏi về quy trình quản lý học liệu điện tử, thuộc phạm vi in-domain của trường UIT.'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về quy trình quản lý học liệu điện tử, thuộc phạm vi in-domain của trường UIT..
Please continue with the current request.
Agent name: Retrieve_Agent
Tôi cần tìm kiếm thông tin trong cơ sở dữ liệu trước khi có thể trả lời câu hỏi của bạn.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'Học liệu điện tử sau khi được thông qua bởi ĐVQLMH có thể bị thẩm định lại lần nữa hay không?'}
Tool output:
 Điều 8. Xây dựng, thẩm định học liệu điện tử, QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 8. Xây dựng, thẩm định học liệu điện tử 1. Học liệu điện tử sử dụng trong DHTT và DHKH phải có nội dung phù hợp với đề 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo điều 8. Xây dựng, thẩm định học liệu điện tử, QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾN VÀ PHƯƠNG THỨC KẾT HỢP CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, các ĐVQLMH có trách nhiệm thẩm định, đảm bảo chất lượng học liệu điện tử phục vụ DHTT, DHKH đối với các môn học mà đơn vị phụ trách. Trường có thể tổ chức thẩm định lại trong trường hợp cần thiết. Bạn hãy yên tâm rằng học liệu của mình sẽ luôn được kiểm duyệt kỹ càng để đảm bảo chất lượng nhé!\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'Retrieve_Agent', 'reason': 'Câu hỏi về quy trình quản lý học liệu điện tử, thuộc phạm vi in-domain của trường UIT.'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về quy trình quản lý học liệu điện tử, thuộc phạm 

In [55]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg=query_in_domain_internet, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'reason': 'Câu hỏi về điểm chuẩn là in-domain.', 'to_agent': 'Retrieve_Agent'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điểm chuẩn là in-domain..
Please continue with the current request.
Agent name: Retrieve_Agent
Tôi sẽ tìm thông tin về điểm chuẩn ngành Khoa học Máy tính năm 2023 trong cơ sở dữ liệu nội bộ của trường.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'Điểm chuẩn ngành Khoa học Máy tính năm 2023'}
Tool output:
 Điều 16. Chấm thi, nhập điểm và công bố điểm, QUY ĐỊNH Tổ chức thi các môn học hệ đại học chính quy của Trường Đại học Công nghệ Thông tin, Điều 16. Chấm thi, nhập điểm và công bố điểm 1. ĐVQLMH của môn học có trách nhiệm tổ chức công tác chấm thi cho môn học mà mình quản lý. Khoa chịu trách nhiệm chính về sự chính xác, công bằng và các khía cạnh chuyên môn khác của việc chấm thi. 2. Đối với kỳ thi cuối học kỳ, 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo trang *truongvietnam.net* (https://truongvietnam.net/diem-chuan-uit/), điểm chuẩn ngành Khoa học Máy tính của UIT năm 2023 là 24.55. Bạn có thể tham khảo thêm thông tin chi tiết trên trang này nhé.\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'reason': 'Câu hỏi về điểm chuẩn là in-domain.', 'to_agent': 'Retrieve_Agent'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điểm chuẩn là in-domain..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'reason': 'Câu hỏi về điểm chuẩn là in-domain.', 'to_agent': 'Retrieve_Agent'}}, raw_output='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điểm chuẩn là in-domain..\nPlease continue w

In [56]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg=query_in_domain_internet_2, stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'reason': 'Câu hỏi liên quan đến thông tin về trường UIT.', 'to_agent': 'Retrieve_Agent'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi liên quan đến thông tin về trường UIT..
Please continue with the current request.
Agent name: Retrieve_Agent

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'Hiệu trưởng trường Đại học Công nghệ Thông tin (UIT) là ai?'}
Tool output:
 Điều 2. Chỉ đạo công tác thi, QUY ĐỊNH Tổ chức thi các môn học hệ đại học chính quy của Trường Đại học Công nghệ Thông tin, Điều 2. Chỉ đạo công tác thi Hiệu trưởng hoặc Phó Hiệu trưởng phụ trách đào tạo đại học trực tiếp chỉ đạo công tác thi. Phòng Đào tạo Đại học (P.ĐTĐH) là đơn vị lập kế hoạch và điều hành công tác tổ chức thi.
Điều 4. Cơ cấu tổ chức, QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG, Điều 4. Cơ cấu tổ chức BĐH cấp trường (sau đây gọi là BĐH) quản lý CTTN dưới sự chỉ đạo

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo doanhnhansaigon.vn (https://doanhnhansaigon.vn/tai-bo-nhiem-ts-nguyen-hoang-tu-anh-lam-hieu-truong-uit-206483.html), TS. Nguyễn Hoàng Tú Anh đã được tái bổ nhiệm làm Hiệu trưởng trường Đại học Công nghệ Thông tin (UIT). Chúc mừng bạn đã có thông tin mới nhất về vị trí hiệu trưởng của trường!\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'reason': 'Câu hỏi liên quan đến thông tin về trường UIT.', 'to_agent': 'Retrieve_Agent'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi liên quan đến thông tin về trường UIT..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'reason': 'Câu hỏi liên quan đến thông tin về trường UIT.', 'to_agent': 'Retrieve_Agent'}}, raw_output='Agent

In [57]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='địa chỉ trường ở đâu', stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'reason': 'Câu hỏi về địa chỉ trường thuộc loại in-domain.', 'to_agent': 'Retrieve_Agent'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về địa chỉ trường thuộc loại in-domain..
Please continue with the current request.
Agent name: Retrieve_Agent
Tôi cần tìm kiếm thông tin về địa chỉ trường trong cơ sở dữ liệu trước khi có thể trả lời bạn.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'địa chỉ trường UIT'}
Tool output:
 Điều 3. Giải thích từ ngữ và chữ viết tắt, QUY ĐỊNH TẠM THỜI VỀ ĐÀO TẠO LIÊN THÔNG TỪ TRÌNH ĐỘ ĐẠI HỌC LÊN TRÌNH ĐỘ THẠC SĨ HỆ CHÍNH QUY, Điều 3. Giải thích từ ngữ và chữ viết tắt Trường: Trường Đại học Công nghệ Thông tin, Đại học Quốc gia Thành phố Hồ Chí Minh. Phòng ĐTĐH: Phòng Đào tạo đại học, Trường Đại học Công nghệ Thông tin. Phòng ĐTSĐH: Phòng Đào tạo Sau đại học và Khoa học Công nghệ, Trường Đại học Công nghệ Thông 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo thông tin từ trang *thongtintuyensinh.vn* (https://www.thongtintuyensinh.vn/Truong_Dai_hoc_Cong_nghe_Thong_tin_DH_Quoc_gia_TPHCM_C51_D822.htm), địa chỉ của Trường Đại học Công nghệ Thông tin là: KP 6, P. Linh Trung, Q. Thủ Đức, TP. Hồ Chí Minh. Đây là địa chỉ chính thức của trường, bạn có thể tham khảo nhé!\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'reason': 'Câu hỏi về địa chỉ trường thuộc loại in-domain.', 'to_agent': 'Retrieve_Agent'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về địa chỉ trường thuộc loại in-domain..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'reason': 'Câu hỏi về địa chỉ trường thuộc loại in-domain.', 'to_agent': 'Retrieve_Agent'}}

In [58]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='bao nhiêu điểm toeic để tốt nghiệp', stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp

Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'reason': 'Câu hỏi về điểm TOEIC để tốt nghiệp thuộc loại in-domain.', 'to_agent': 'Retrieve_Agent'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điểm TOEIC để tốt nghiệp thuộc loại in-domain..
Please continue with the current request.
Agent name: Retrieve_Agent
Tôi cần tìm kiếm thông tin trong cơ sở dữ liệu trước khi có thể trả lời bạn.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'điểm TOEIC để tốt nghiệp'}
Tool output:
 Điều 33. Điều kiện xét tốt nghiệp và công nhận tốt nghiệp, QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 33. Điều kiện xét tốt nghiệp và công nhận tốt nghiệp 1. Mỗi năm Trường có 4 đợt xét tốt nghiệp và cấp bằng tốt nghiệp đại học, lịch xét tốt nghiệp được ghi trong kế hoạch năm học. Để được Trường xét cấp bằng tốt nghiệp, sinh viên nộp hồ sơ đề nghị xét tố

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp, QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, đối với chương trình chuẩn, bạn cần đạt chứng chỉ tiếng Anh TOEIC Nghe-Đọc đạt từ 450 trở lên và TOEIC Nói-Viết đạt từ 185 trở lên. Hãy cố gắng học tập và rèn luyện để đạt được kết quả tốt nhất nhé!\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'reason': 'Câu hỏi về điểm TOEIC để tốt nghiệp thuộc loại in-domain.', 'to_agent': 'Retrieve_Agent'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điểm TOEIC để tốt nghiệp thuộc loại in-domain..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwar

In [59]:
# model: gemini-2.0-flash-001

# Comment: Retrieve_Agent now does not follow its system prompt
# This is a unexpected case when changing user_msg from 'Hiệu trưởng UIT là ai?' (query_in_domain_internet_2) to 'hiệu trưởng là ai' (this cell)

handler = multi_agents.run(user_msg='hiệu trưởng là ai', stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp

Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'reason': 'Câu hỏi về hiệu trưởng trường UIT', 'to_agent': 'Retrieve_Agent'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về hiệu trưởng trường UIT.
Please continue with the current request.
Agent name: Retrieve_Agent
Tôi xin lỗi, tôi không thể trả lời câu hỏi này. Để tôi chuyển câu hỏi này cho bộ phận hỗ trợ tìm kiếm thông tin để được hỗ trợ tốt nhất.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Tôi xin lỗi, tôi không thể trả lời câu hỏi này. Để tôi chuyển câu hỏi này cho bộ phận hỗ trợ tìm kiếm thông tin để được hỗ trợ tốt nhất.\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'reason': 'Câu hỏi về hiệu trưởng trường UIT', 'to_agent': 'Retrieve_Agent'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về hiệu trưởng trường UIT.\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'reason': 'Câu hỏi về hiệu trưởng trường UIT', 'to_agent': 'Retrieve_Agent'}}, raw_output='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về hiệu trưởng trường UIT.\nPlease continue with the current request.', is_error=False), return_direct=True)], raw={'co

**LLM is one of the most factors that can improve your agents**

* Benefit: larger LLM, better reasoning
* Drawback: slower inference, reduces user's experience

*(individual thought): try gemeni-flash-2.0 can help us write good system prompt first to cover most cases, then we change to gemini-2.5-flash which help to fix special case that gemeni-flash-2.0 cannot well handle*

In [42]:
# model: gemini-2.5-flash-preview-04-17
# DO NOT forget to change the model and run related multi-agent cells

handler = multi_agents.run(user_msg='hiệu trưởng là ai', stream=True)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'to_agent': 'Retrieve_Agent', 'reason': 'Câu hỏi in-domain, cần Retrieve_Agent để trả lời thông tin về hiệu trưởng.'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi in-domain, cần Retrieve_Agent để trả lời thông tin về hiệu trưởng..
Please continue with the current request.
Agent name: Retrieve_Agent

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'hiệu trưởng là ai'}
Tool output:
 Điều 4. Cơ cấu tổ chức, QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG, Điều 4. Cơ cấu tổ chức BĐH cấp trường (sau đây gọi là BĐH) quản lý CTTN dưới sự chỉ đạo của Hiệu trưởng. BĐH do Hiệu trưởng ký quyết định thành lập, bao gồm: - Hiệu trưởng hoặc Phó Hiệu trưởng phụ trách đào tạo làm Trưởng Ban; - Trưởng đơn vị quản lý chương trình làm Ủy viên thường trực; - Đại diện các Khoa mở ngành đào tạo CTTN;
Điều 2. Phân cấp quản lý công tác giáo trình, QUI ĐỊNH VỀ CÔNG TÁC GIÁO TR

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo thông tin từ trang web chính thức của Trường Đại học Công nghệ Thông tin (UIT) về Ban Giám hiệu (https://www.uit.edu.vn/ban-giam-hieu) và bài viết trên báo Tuổi Trẻ (https://tuoitre.vn/hieu-truong-truong-dai-hoc-cong-nghe-thong-tin-khuyen-sinh-vien-dung-chim-dam-trong-mang-xa-hoi-20240905212811507.htm), Hiệu trưởng hiện tại của Trường Đại học Công nghệ Thông tin là **PGS.TS Nguyễn Hoàng Tú Anh**.')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'Retrieve_Agent', 'reason': 'Câu hỏi in-domain, cần Retrieve_Agent để trả lời thông tin về hiệu trưởng.'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi in-domain, cần Retrieve_Agent để trả lời thông tin về hiệu trưởng..\nPlease continue with the current request.', too

# Context

https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/memory/ <br>
https://docs.llamaindex.ai/en/stable/understanding/agent/state/ <br>
https://docs.llamaindex.ai/en/stable/understanding/agent/multi_agent/

In [57]:
from llama_index.core.workflow import Context
ctx = Context(multi_agents)

In [ ]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='cần bao nhiêu điểm toeic thì tốt nghiệp', stream=True, ctx=ctx)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Retrieve_Agent

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'điểm TOEIC cần thiết để tốt nghiệp UIT'}
Tool output:
 Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp, QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp Để được xét tốt nghiệp sinh viên phải hoàn tất các môn học tiếng Anh trong chương trình giảng dạy tiếng Anh tại Điều 3 và có một trong các văn bằng, chứng chỉ/chứng nhận được quy định như sau: 1. Văn bằng tốt nghiệp đại học/trên đại học trong nước, nước ngoài mà ngành học hoặc ngôn ngữ sử dụng toàn phần trong đào tạo (không thông qua phiên dịch) là một trong ba ngoại ngữ: tiếng Anh, tiếng Pháp, tiếng Nhật. 2. Chứng chỉ/chứng nhận ngoại ngữ: Sinh viên có một trong các loại chứng chỉ/chứng nhận tiếng Anh, có chứng chỉ/chứng nhận tiếng Pháp, hoặc tiếng Nhật tối thiểu nh

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo Điều 9 của QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, điểm TOEIC cần thiết để tốt nghiệp tùy thuộc vào chương trình bạn theo học:\n\n*   **Chương trình chuẩn (văn bằng 2, liên thông):** TOEIC Nghe-Đọc từ 450 trở lên và TOEIC Nói-Viết từ 185 trở lên.\n*   **Chương trình tài năng, chất lượng cao:** TOEIC Nghe-Đọc từ 555 trở lên và TOEIC Nói-Viết từ 205 trở lên.\n*   **Chương trình tiên tiến:** TOEIC Nghe-Đọc từ 675 trở lên và TOEIC Nói-Viết từ 205 trở lên.\n\nBạn hãy xác định chương trình mình đang theo học và cố gắng đạt số điểm TOEIC tương ứng nhé!')]), tool_calls=[ToolCallResult(tool_name='retrieve_database', tool_kwargs={'query': 'điểm TOEIC cần thiết để tốt nghiệp UIT'}, tool_id='retrieve_database', tool_output=ToolOutput(content='Điều 9. Văn bằng, chứng chỉ/chứng nhận đư

In [ ]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='địa chỉ trường ở đâu', stream=True, ctx=ctx)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Retrieve_Agent

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'Địa chỉ của trường Đại học Công nghệ Thông tin'}
Tool output:
 Điều 3. Giải thích từ ngữ và chữ viết tắt, QUY ĐỊNH TẠM THỜI VỀ ĐÀO TẠO LIÊN THÔNG TỪ TRÌNH ĐỘ ĐẠI HỌC LÊN TRÌNH ĐỘ THẠC SĨ HỆ CHÍNH QUY, Điều 3. Giải thích từ ngữ và chữ viết tắt Trường: Trường Đại học Công nghệ Thông tin, Đại học Quốc gia Thành phố Hồ Chí Minh. Phòng ĐTĐH: Phòng Đào tạo đại học, Trường Đại học Công nghệ Thông tin. Phòng ĐTSĐH: Phòng Đào tạo Sau đại học và Khoa học Công nghệ, Trường Đại học Công nghệ Thông tin. Phòng DL&CNTT: Phòng Dữ liệu và Công nghệ Thông tin Ngành ĐH-ThS: Ngành của chương trình ĐH-ThS. Chương trình/Ngành ĐH đang học: Chương trình/Ngành học trình độ đại học của sinh viên khi dự tuyển vào chương trình ĐH-ThS. Người học: sinh viên, học viên tham gia chương trình ĐH-ThS.
Điều 25. Cách tính điểm trung bình, QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo \\`tuoitre.vn\\` ([https://tuoitre.vn/truong-dai-hoc-cong-nghe-thong-tin-nhung-buoc-di-tien-phong-sang-tao-20200604104917867.htm](https://tuoitre.vn/truong-dai-hoc-cong-nghe-thong-tin-nhung-buoc-di-tien-phong-sang-tao-20200604104917867.htm)) và \\`citd.vn\\` ([https://www.citd.vn/sodo_uit/](https://www.citd.vn/sodo_uit/)), địa chỉ của Trường Đại học Công nghệ Thông tin là:\n\n**Khu phố 6, Phường Linh Trung, Thành phố Thủ Đức, Thành phố Hồ Chí Minh.**\n\nĐây là địa chỉ chính thức của trường UIT, bạn nhé!')]), tool_calls=[ToolCallResult(tool_name='retrieve_database', tool_kwargs={'query': 'Địa chỉ của trường Đại học Công nghệ Thông tin'}, tool_id='retrieve_database', tool_output=ToolOutput(content='Điều 3. Giải thích từ ngữ và chữ viết tắt, QUY ĐỊNH TẠM THỜI VỀ ĐÀO TẠO LIÊN THÔNG TỪ TRÌNH ĐỘ ĐẠI HỌC LÊN TRÌNH ĐỘ THẠC SĨ HỆ CHÍNH

In [ ]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='á đù', stream=True, ctx=ctx)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Search_Agent
Tôi xin lỗi nếu câu trả lời của tôi không phù hợp với mong đợi của bạn. Nếu bạn có bất kỳ câu hỏi nào khác, đừng ngần ngại hỏi nhé!


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Tôi xin lỗi nếu câu trả lời của tôi không phù hợp với mong đợi của bạn. Nếu bạn có bất kỳ câu hỏi nào khác, đừng ngần ngại hỏi nhé!\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' bạn có bất kỳ câu hỏi nào khác, đừng ngần ngại hỏi nhé!\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 36, 'candidates_tokens_details': [{'modal

Multi-agent with `ctx` does not follow workflow: filter_agent -> retreive_agent -> search_agent <br>
=> **`ctx` breaks the defined workflow**

In [45]:
# Get key and value of the context
for key in ctx.data.keys():
    value = await ctx.get(key)
    print(f"{key}: {value}")

memory: chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='cần bao nhiêu điểm toeic thì tốt nghiệp')]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': [FunctionCall(id=None, args={'to_agent': 'Retrieve_Agent', 'reason': 'Câu hỏi về điều kiện tốt nghiệp, thuộc phạm vi in-domain của trường UIT.'}, name='handoff')]}, blocks=[TextBlock(block_type='text', text='')]), ChatMessage(role=<MessageRole.TOOL: 'tool'>, additional_kwargs={'tool_call_id': 'handoff'}, blocks=[TextBlock(block_type='text', text='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điều kiện tốt nghiệp, thuộc phạm vi in-domain của trường UIT..\nPlease continue with the current request.')]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': [FunctionCall(id=None, args={'query': 'điểm TOEIC cần thiết đ

C:\Users\Dell\AppData\Local\Temp\ipykernel_6476\2964164002.py:2: DeprecationWarning: `data` is deprecated, please use the `get` and `set` method to store data into the Context.
  for key in ctx.data.keys():


**Benefit**
* Explicit Control: By resetting the current_agent_name, you ensure the workflow starts with the correct agent every time.
* Preserves Context: You can still use ctx to maintain other state variables across transactions while controlling agent routing.
* Avoids Fresh Context Creation: This approach avoids creating a new Context for each query, which might be inefficient if you need to preserve other state variables.

**Trade-offs**
* Manual Reset: Requires explicit resetting of the agent state before each query, which adds some overhead.
* Potential for Errors: If the reset is missed, the workflow may still route incorrectly.

**>>> run method arguments**:

* **ctx (Context)**: Stores the overall workflow state, including `memory` and other variables, and persists information across multiple runs. Use it to maintain long-term conversational context and agent state.

* **memory**: A stateful object (e.g., ChatMemoryBuffer) that manages, updates, and retrieves chat history during a run. It enables dynamic, persistent memory strategies.

* **chat_history**: A static list of messages used as context for a single run. It does not persist or update after the run—ideal for one-off overrides or initializing memory.

In [ ]:
# get chat_history from memory

# memory = await ctx.get("memory")
# chat_history = memory.get_all()
# chat_history

let's use `ctx` with reseting `current_agent_name` each time run method executed <br>

**Update**: I already try: `ctx.set('current_agent_name', 'Filter_Agent')` but the multi-agent still does not follow the defined workflow

=> try to use `ChatMemoryBuffer`

In [44]:
# As the agent runs, it will make calls to memory.put() to store information, and memory.get() to retrieve information.

from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=40000)

In [ ]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='cần bao bn điểm toeic để tốt nghiệp', stream=True, memory=memory)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'reason': 'Câu hỏi về điều kiện tốt nghiệp (TOEIC) của trường UIT.', 'to_agent': 'Retrieve_Agent'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điều kiện tốt nghiệp (TOEIC) của trường UIT..
Please continue with the current request.
Agent name: Retrieve_Agent
Tôi cần tìm kiếm thông tin trong cơ sở dữ liệu trước khi có thể trả lời bạn.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'điểm TOEIC để tốt nghiệp UIT'}
Tool output:
 Điều 13. Tốt nghiệp, QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TIÊN TIẾN, Điều 13. Tốt nghiệp Sinh viên được xét công nhận tốt nghiệp khi đạt tất cả các yêu cầu được quy định trong quy chế đào tạo theo học chế tín chỉ của Trường. Sinh viên được cấp bằng “Kỹ sư Chương trình Tiên tiến” (Advanced Program) của Trường ĐH CNTT – ĐHQG HCM.
Điều 33. Điều kiện xét tốt nghiệp và công nhận tốt nghiệp, QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ C

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp, QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, đối với chương trình văn bằng 2, liên thông, chương trình chuẩn phải đạt chứng chỉ tiếng Anh TOEIC Nghe-Đọc đạt từ 450 trở lên và TOEIC Nói-Viết đạt từ 185 trở lên hoặc TOEFL iBT đạt từ 45 trở lên hoặc IELTS đạt 4.5, chứng chỉ Cambridge English A2 Key (Grade A) hoặc B1 Preliminary (Grade C) hoặc B1 Business Preliminary (Pass), chứng chỉ VNU-EPT đạt từ 176 trở lên, khung chứng chỉ theo năng lực 6 bậc đạt từ B1. Bạn hãy cố gắng học tập để đạt được kết quả tốt nhất nhé!\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'reason': 'Câu hỏi về điều kiện tốt nghiệp (TOEIC) của trường UIT.', 'to_agent': 'Retrieve_Agent'}, tool_id='ha

In [ ]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='nếu tôi thi được 450 toeic nghe đọc thì đủ điều kiện chưa?', stream=True, memory=memory)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'to_agent': 'Retrieve_Agent', 'reason': 'Câu hỏi về điều kiện tốt nghiệp (TOEIC) của trường UIT.'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điều kiện tốt nghiệp (TOEIC) của trường UIT..
Please continue with the current request.
Agent name: Retrieve_Agent
Để tôi kiểm tra thông tin này trong cơ sở dữ liệu nhé.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'điều kiện toeic 450 nghe đọc tốt nghiệp UIT'}
Tool output:
 Điều 33. Điều kiện xét tốt nghiệp và công nhận tốt nghiệp, QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 33. Điều kiện xét tốt nghiệp và công nhận tốt nghiệp 1. Mỗi năm Trường có 4 đợt xét tốt nghiệp và cấp bằng tốt nghiệp đại học, lịch xét tốt nghiệp được ghi trong kế hoạch năm học. Để được Trường xét cấp bằng tốt nghiệp, sinh viên nộp hồ sơ đề nghị xét tốt nghiệ

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo điều 9. Văn bằng, chứng chỉ/chứng nhận được sử dụng để công nhận đạt chuẩn xét tốt nghiệp, QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, đối với chương trình văn bằng 2, liên thông, chương trình chuẩn, bạn cần đạt đồng thời TOEIC Nghe-Đọc từ 450 trở lên **VÀ** TOEIC Nói-Viết đạt từ 185 trở lên. Nếu bạn chỉ đạt 450 TOEIC Nghe-Đọc thì chưa đủ điều kiện tốt nghiệp. Bạn cần cố gắng hơn nữa nhé!\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'Retrieve_Agent', 'reason': 'Câu hỏi về điều kiện tốt nghiệp (TOEIC) của trường UIT.'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về điều kiện tốt nghiệp (TOEIC) của trường UIT..\nPlease continue with the current reques

> multi-agent now can follow up the context in the past and it also adapt, connect to query at present althouth the query *"nếu tôi thi được 450 toeic nghe đọc thì đủ điều kiện chưa?"* does not refer to the graduation, and the result returned from the tool retrieve_database does not have "Điều 9" which is condition for graduation with TOEIC English score at 4 skills

NOTE: When you rerun the cell above, Agent can refine your query (convert user's style to Agent's style that it thinks better one), and the retreival result is not consistent (in my turn, "Điều 9" does not exist in returned doc)

The only issue here, which is **unanticipated Retreive_agent's behavior**, is Retreive_agent automatically refines user query which leads to the bad or good result returned by the tool retrieve_database

In [ ]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='địa chỉ trường ở đâu', stream=True, memory=memory)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent

Tool called:  handoff
Arguments to the tool:  {'reason': 'Câu hỏi về địa chỉ của trường UIT.', 'to_agent': 'Retrieve_Agent'}
Tool output:
 Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về địa chỉ của trường UIT..
Please continue with the current request.
Agent name: Retrieve_Agent
Để tôi tìm kiếm thông tin này trong cơ sở dữ liệu nhé.

Tool called:  retrieve_database
Arguments to the tool:  {'query': 'địa chỉ trường Đại học Công nghệ Thông tin'}
Tool output:
 Điều 30. Công bố thông tin, QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠI HỌC CHÍNH QUY CỦA TRƯỜNG ĐẠI HỌC CÔNG NGHỆ THÔNG TIN, Điều 30. Công bố thông tin 1. Trường xây dựng trang thông tin điện tử để công bố công khai chuẩn đầu ra, nội dung chương trình đào tạo, điều kiện đảm bảo chất lượng đào tạo tương ứng với quy mô theo chỉ tiêu hàng năm, quy định học vụ và cho phép người sử dụng tra cứu thông tin cơ bản của sinh viên tốt nghiệp theo số hiệu tương ứng trên vă

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Theo trang web chính thức của Trường Đại học Công nghệ Thông tin (https://www.uit.edu.vn/lien-he), địa chỉ của trường là: **Khu phố 6, phường Linh Trung, Quận Thủ Đức, Tp. Hồ Chí Minh**. Bạn lưu ý địa chỉ này nhé!\n')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'reason': 'Câu hỏi về địa chỉ của trường UIT.', 'to_agent': 'Retrieve_Agent'}, tool_id='handoff', tool_output=ToolOutput(content='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về địa chỉ của trường UIT..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'reason': 'Câu hỏi về địa chỉ của trường UIT.', 'to_agent': 'Retrieve_Agent'}}, raw_output='Agent Retrieve_Agent is now handling the request due to the following reason: Câu hỏi về địa chỉ của trường UIT..\nPlease con

In [ ]:
# model: gemini-2.0-flash-001

handler = multi_agents.run(user_msg='á đù', stream=True, memory=memory)
async for event in handler.stream_events():
    if isinstance(event, ToolCallResult):
        print("")
        print("Tool called: ", event.tool_name)              # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output:\n", event.tool_output)           # the tool output
        
    elif isinstance(event, AgentStream):                     
        print(event.delta, end="", flush=True)               # showing the thought process
        
    elif isinstance(event, AgentInput):                    
        print("Agent name:", event.current_agent_name)      # the current agent name


resp = await handler
resp


Agent name: Filter_Agent
Câu hỏi của bạn có vẻ như là lạm dụng hệ thống của chúng tôi. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Câu hỏi của bạn có vẻ như là lạm dụng hệ thống của chúng tôi. Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.\n')]), tool_calls=[], raw={'content': {'parts': [{'video_metadata': None, 'thought': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_response': None, 'inline_data': None, 'text': ' Bạn vui lòng đặt câu hỏi khác để nhận được trợ giúp.\n'}], 'role': 'model'}, 'citation_metadata': None, 'finish_message': None, 'token_count': None, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'avg_logprobs': None, 'grounding_metadata': None, 'index': None, 'logprobs_result': None, 'safety_ratings': None, 'usage_metadata': {'cache_tokens_details': None, 'cached_content_token_count': None, 'candidates_token_count': 31, 'candidates_tokens_details': [{'modality': <MediaModality

# Improvement in future

* **retreival performance**: 
	1. using [filter](https://docs.llamaindex.ai/en/stable/examples/vector_stores/Qdrant_using_qdrant_filters/) when searching database (mannually or leverage LLM to indicate topic for each doc in corpus), here we just reuse everyhing in previous project and metadata maybe not useful (recommended)
	2. create larger dataset with AI-generated questions to make question-document pairs more diverse
	3. extra techinques: query refinement, requery expansion, multi-step query, ...
	4. training sentence transforemer model (bi-encoder): balance speed and accuracuracy with [Matryoshka Embeddings](https://sbert.net/examples/sentence_transformer/training/matryoshka/README.html) or using model with longer context window
	5. reranking: use cross-encoder (reranker) to re-rank top-k results returned by bi-encoder (recommended)
	6. change to another vector database (Ex: Chorma)

* **agentic workflow**: provide more tools, add more agents and desingn an effective and efficient structure of multi-agent system (for example, adding supervisor agent or reducing tool use frequent - in this project, each query is used to retrieve internal database)

* **prompt design**: write better, consistent prompts to help agent to be more confident and always obey the defined rules

* make [memory](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/memory/) better

